In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import re
from pathlib import Path 

import tensorflow as tf
import tensorflow.keras.preprocessing.text as Text
import tensorflow.keras.preprocessing.sequence as Sequence

In [2]:
df = pd.read_pickle('./data/two-topic-fulltext.pkl')

def clean_text(text):
    """
    Set text to lowercase and replace all non-ascii characters with spaces.
    """
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    return text

df["full_text"] = df["full_text"].apply(lambda x: clean_text(x))

# If tokenized text sequences exist, load as X, else tokenize the dataset and save.
my_file = Path("./data/text-sequences.npy")

if my_file.is_file():
    X = np.load("./data/text-sequences.npy")
else:
    tokenizer = Text.Tokenizer(num_words=20000,
                               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                               lower=True, split=' ', char_level=False,
                               oov_token=None)
    tokenizer.fit_on_texts(df["full_text"].values)

    index_words = tokenizer.word_index
    print(f"Number of words: {len(index_words)}")

    X = tokenizer.texts_to_sequences(df["full_text"].values)

    X = Sequence.pad_sequences(X, maxlen=250)

    np.save("./data/text-sequences.npy", X)

In [3]:
Y = df["topic"].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=123)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
callbacks = [EarlyStopping(monitor='val_acc', min_delta=.0001, patience=2, verbose=True)]

model = Sequential()
model.add(layers.Embedding(20_000, 100, input_length=X.shape[1]))
model.add(layers.LSTM(100, dropout=0.25, recurrent_dropout=0.25))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

W0515 20:58:20.013479 139839040452416 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f2cf104ff60>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [ ]:
history = model.fit(X_train, Y_train, epochs=4, batch_size=32, validation_split=0.1, callbacks=callbacks)

Train on 15300 samples, validate on 1700 samples
Epoch 1/4
15296/15300 [============================>.] - ETA: 0s - loss: 0.5323 - accuracy: 0.7524

W0515 21:10:02.008979 139839040452416 callbacks.py:1004] Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


15300/15300 [==============================] - 700s 46ms/sample - loss: 0.5323 - accuracy: 0.7525 - val_loss: 0.4314 - val_accuracy: 0.8282
Epoch 2/4
 8864/15300 [================>.............] - ETA: 4:57 - loss: 0.3845 - accuracy: 0.8413

In [ ]:
model.evaluate(X_test,Y_test)

In [ ]:
model.save("paper_classifier2.h5")

In [9]:
# model = tf.keras.models.load_model("./paper_classifier2.h5")
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn-deep')

plt.figure(figsize=(5, 4))
plt.title("Accuracy")
plt.plot(history.history["accuracy"], label='train')
plt.plot(history.history["val_accuracy"], label='test')
plt.legend()
plt.show()
plt.savefig("accuracy-text-classifier.png")

plt.figure(figsize=(5, 4))
plt.title("Loss")
plt.plot(history.history["loss"], label='train')
plt.plot(history.history["val_loss"], label='test')
plt.legend()
plt.show()
plt.savefig("loss-text-classifier.png")

In [7]:
Y_test_predict = model.predict(X_test)

plt.figure(figsize=(5, 4))
plt.hist(Y_test_predict[Y_test == 0], alpha = 0.5)
plt.hist(Y_test_predict[Y_test == 1], alpha = 0.5)
plt.legend(["Deep Learning", "Computer Vision"])
plt.title("Prediction Confidence for Both Topics")
plt.show()
plt.savefig("./results/prediction-confidences.png")

In [65]:
uncertain_predictions_mask = (Y_test_predict > 0.4) & (Y_test_predict < 0.6)
X_uncertain = X_test[uncertain_predictions_mask.flatten()]

df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df["full_text"].values, df["topic"].values,
                                                                test_size=0.15, random_state=123)

uncertain_papers = df_x_test[uncertain_predictions_mask.flatten()]
print("Full text of uncertain prediction:")
uncertain_papers[1]

Full text of uncertain prediction:


'multiple kernel learning for brain-computer interfacing wojciech samek1, alexander binder1, klaus-robert muller1,2 abstract combining information from different sources is a common way to improve classification accuracy in brain- computer interfacing (bci). for instance, in small sample set- tings it is useful to integrate data from other subjects or sessions in order to improve the estimation quality of the spatial filters or the classifier. since data from different subjects may show large variability, it is crucial to weight the contributions according to importance. many multi-subject learning algorithms determine the optimal weighting in a separate step by using heuristics, however, without ensuring that the selected weights are optimal with respect to classification. in this work we apply multiple kernel learning (mkl) to this problem. mkl has been widely used for feature fusion in computer vision and allows to simultaneously learn the classifier and the optimal weighting. we co

In [58]:
test_loss = model.evaluate(X_test, Y_test)

3000/3000 [==============================] - 15s 5ms/sample - loss: 0.8814 - accuracy: 0.8203
